<a href="https://colab.research.google.com/github/raheelam98/LangChain_Fundamentals/blob/main/langchain_course_jupyter/section_4_re_act_agent/part1_re_act_agent_tools.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Section 5: ReAct Agent Tools**

[Defining Custom Tools](https://python.langchain.com/v0.1/docs/modules/tools/custom_tools/)



## **Defining Tools for ReAct agent**

In [ ]:
# Install the required packages:
%%capture --no-stderr
%pip install --quiet -U python-dotenv langsmith langchain langchain_google_genai

In [ ]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "ReAct Agent"

In [ ]:
# API Keys
# Get the GEMINI API key from user data
from google.colab import userdata
gemini_api_key = userdata.get('GEMINI_API_KEY')

In [ ]:
# llm
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    max_retries=2,
    api_key=gemini_api_key
)

In [ ]:
# app/main.py

from dotenv import load_dotenv

load_dotenv()

def get_text_length_test(text:str)-> int:
  """Returns the length of a text by characters"""
  print(f"get_text_length enter with {text=}")
  text = text.strip("'\n").strip(
      '"'
  ) # stripping away non alphabetic character just in case

  return len(text)


In [ ]:
print("ReAct Agent")
print(get_text_length_test(text="ReAct Agent"))

ReAct Agent
get_text_length enter with text='ReAct Agent'
11


#### **Defining Tools for ReAct agent Part1(Detail)**

[LangChain Hub](https://smith.langchain.com/hub?organizationId=607e9d67-1fc3-4433-bc0c-4507b1714844)

[How to create tools](https://python.langchain.com/docs/how_to/custom_tools/)

In [ ]:
# app/main.py

from langchain.tools import tool
from dotenv import load_dotenv

load_dotenv()

@tool
def get_text_length(text:str) -> int:
  """Returns the length of a text by characters"""

  return len(text)


In [ ]:
print("\nReAct Agent\n")
tools = [get_text_length]

template= """
Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}
"""


ReAct Agent



**PromptTemplate**

```
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(template=template).partial()
```


In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(template=template).partial(
    tools=tools,
    tool_names= ",".join([t.name for t in tools])
    )

# print(prompt)

**Render the tool name and description in plain text.**

```
from langchain.tools.render import render_text_description

render_text_description()
```

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.tools.render import render_text_description

prompt = PromptTemplate.from_template(template=template).partial(
    tools=render_text_description(tools),
    tool_names= ",".join([t.name for t in tools])
    )


In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
# Initialize the ChatGoogleGenerativeAI model
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",  # Specify the model
    api_key=gemini_api_key,  # Pass the API key
    stop = ["\nObservation"]
    )

In [ ]:
res = llm.invoke("hi")
print(res)

content='Hi there! How can I help you today?' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-8c74b00e-c168-4d59-a094-46a6d2025f06-0' usage_metadata={'input_tokens': 2, 'output_tokens': 11, 'total_tokens': 13, 'input_token_details': {'cache_read': 0}}


ReAct prompt, LLM Reasoning Engine, **Output Parsing and Tool** Execution

[LangChain Expression Language (LCEL)](https://python.langchain.com/docs/concepts/lcel/)

#### **Defining Tools for ReAct agent - Part 1**

In [ ]:
## Part 1 : Defining Tools for our ReAct agent
## 28. Defining Tools for our ReAct agent

from typing import Union, List

from dotenv import load_dotenv
# from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.schema import AgentAction, AgentFinish
from langchain.tools import Tool, tool
from langchain.tools.render import render_text_description

load_dotenv()

@tool
def get_text_length(text: str) -> int:
    """Returns the length of a text by characters"""
    print(f"get_text_length enter with {text=}")
    text = text.strip("'\n").strip(
        '"'
    )  # stripping away non alphabetic characters just in case

    return len(text)

def find_tool_by_name(tools: List[Tool], tool_name: str) -> Tool:
    for tool in tools:
        if tool.name == tool_name:
            return tool
    raise ValueError(f"Tool wtih name {tool_name} not found")


if __name__ == "__main__":
    print("Hello ReAct LangChain!")
    tools = [get_text_length]

    template = """
    Answer the following questions as best you can. You have access to the following tools:

    {tools}

    Use the following format:

    Question: the input question you must answer
    Thought: you should always think about what to do
    Action: the action to take, should be one of [{tool_names}]
    Action Input: the input to the action
    Observation: the result of the action
    ... (this Thought/Action/Action Input/Observation can repeat N times)
    Thought: I now know the final answer
    Final Answer: the final answer to the original input question

    Begin!

    Question: {input}
    Thought:
    """

    prompt = PromptTemplate.from_template(template=template).partial(
        tools=render_text_description(tools),
        tool_names=", ".join([t.name for t in tools]),
    )

    llm = ChatGoogleGenerativeAI(
                                 model="gemini-1.5-flash",  # Specify the model
                                 api_key=gemini_api_key,  # Pass the API key
                                 stop = ["\nObservation", "Observation"]
                                 )


Hello ReAct LangChain!
